As mentioned in anotehr file, i cannot upload my faiss index to drive

FAISS indexes contain native C++ state and cannot be reliably serialized with pickle. Indexes hebce, are rebuilt from saved embeddings instead of being loaded from disk.


In [2]:
import pickle
PATH_TO_CHUNKS = '/content/drive/MyDrive/colab_stuff/court_hearings_rag/chunk_text.pkl'
with open(PATH_TO_CHUNKS, 'rb') as f:
    metadata = pickle.load(f)

In [3]:
import numpy as np
embedding_bns = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns_embed.npy')
embedding_apex = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/apex_embed.npy')
embedding_high = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_embed.npy')

again reminder, faiss cpu us used because model is going to be too big to leave any space in GPU. Either way, faiss is quite old thing, it was designed around cpu, its fine, its fast, its default so much so i need to manually make gpu element and copy index there faiss.index_cpu_to_gpu()

In [4]:
!pip install faiss-cpu
import faiss

dim = embedding_bns.shape[1]

index_bns = faiss.IndexFlatIP(dim)
index_apex = faiss.IndexFlatIP(dim)
index_high = faiss.IndexFlatIP(dim)

index_bns.add(embedding_bns)
index_apex.add(embedding_apex)
index_high.add(embedding_high)

In [5]:
# validating sizes just confirming its same accross file
print(f'size of bns metadata :: {len(metadata[0])} size of index :: {len(embedding_bns)}')
print(f'size of apex metadata :: {len(metadata[1])} size of index :: {len(embedding_apex)}')
print(f'size of high metadata :: {len(metadata[2])} size of index :: {len(embedding_high)}')

size of bns metadata :: 471 size of index :: 471
size of apex metadata :: 770935 size of index :: 770935
size of high metadata :: 441735 size of index :: 441735


In [6]:
import sentence_transformers
model_faiss = sentence_transformers.SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def search_info(query, model, metadata, top_k=5):
    # first use the same sentance embedding model to make query database compatible
    query_vector = model.encode(query, normalize_embeddings=True, convert_to_numpy=True)

    sim_1, idx_1 = index_bns.search(query_vector, top_k)
    sim_2, idx_2 = index_apex.search(query_vector, top_k)
    sim_3, idx_3 = index_high.search(query_vector, top_k )

    bns, apex, high = metadata
    print(len(query))
    for q in range(len(query)):
        for i in range(top_k):
            print(f'{i+1}th most similar to query in bns with similarity :: {sim_1[q][i]} is :: \n{bns[idx_1[q][i]]}')
            print(f'{i+1}th most similar to query in apex with similarity :: {sim_2[q][i]} is :: \n{apex[idx_2[q][i]]}')
            print(f'{i+1}th most similar to query in high with similarity :: {sim_3[q][i]} is :: \n{high[idx_3[q][i]]}')

query = [
    "What constitutes giving false evidence in a judicial proceeding under criminal law?",
    "What is the punishment for using fabricated or false evidence in a criminal case?"
  ]
search_info(query, model_faiss, metadata, 2)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


2
1th most similar to query in bns with similarity :: 0.6652504205703735 is :: 
{'chunk_id': 'BNS_BNS_228_0', 'chunk_text': 'Whoever causes any circumstance to exist or makes any false entry in any book or record, or electronic record or makes any document or electronic record containing a false statement, intending that such circumstance, false entry or false statement may appear in evidence in a judicial proceeding, or in a proceeding taken by law before a public servant as such, or before an arbitrator, and that such circumstance, false entry or false statement, so appearing in evidence, may cause any person who in such proceeding is to form an opinion upon the evidence, to entertain an erroneous opinion touching any point material to the result of such proceeding is said "to fabricate false evidence".\nIllustrations.\n(a) A puts jewels into a box belonging to Z, with the intention that they may be found in that box, and that this circumstance may cause Z to be convicted of theft. A

Well it seems colab refuses to believe i have bitsandbytes until i restart kernel, so do that after installing them.

BitsAndBytes is responsible for quantization. Model can and does fit in gpu memory even without quantization, but when dealing with large number of documents for retrieval, it gives OOM error.

So i quantized for it. Though i since simplified(considerably dumbed down and reduced) total documents retrievals for generation

In [ ]:
!pip install -U transformers bitsandbytes

from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


Well but bear in mind mistral was trained to be helpful to user, to it talks a lot, can't exactly expect it to shut up as we want, even if it does it still talks sometimes cuz query it is going to get are law/crime related so it tries to clear itself by giving some reasoning or advice, so just accept it. Its unexpectedly time consuming to write prompt to get it to do what i want, no more, no less

In [9]:
import transformers
import torch
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # automatically does cpu/gpu
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True # use rust backend when available
)
model.eval() # it is all pytorch afterall.
print(model.device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

cuda:0


Well i tried with rewriting prompt in an attmpt to improve retrieval, but in most cases the rewritten prompt ends up getting even worse Similarity score than if i sent user query to faiss directly. So that is what i am gonna do, no rewriting.

The reason for this, what i believe, is mistral trying to be overly helpful. It rewrites the prompt but the rewritten one(s) usually sound much much more formal with a very formal english. And i can't seem to be able to stop it from being overly helpful, even with few example prompts given, it does improve much more(few shot learning) but not for all types of question(s) or all type of framing i expect it to handle. I think it cannot be tuned very much, if i was to give an analogy, i need tuning in scale of milimeter and mistral works in realm of centimeter(s), either over or under following the instruction(s).

Though one place where it did worked phenomenal was when user were to share his/her story instead of asking question directly, maybe because user isn't able to pin point as to what exactly is it that they need to ask. It was good at summarizing the story to a sentance or two which can be relatively easily searched in faiss with decent score.


In [36]:
from enum import IntEnum

class Document(IntEnum):
    BNS = 0
    APEX = 1
    HIGH = 2
    ALL = 3

index_map = {
    Document.BNS: index_bns,
    Document.APEX: index_apex,
    Document.HIGH: index_high,
}

In [51]:
def faiss_search(query, model, document, metadata, top_k=2):
    '''
    apex high or bns in document argument to search only one of them
    '''
    if isinstance(query, str):
        query = [query] # we need list for faiss

    query_encoded = model.encode(query, normalize_embeddings=True, convert_to_numpy=True)

    documents_retrieved = []
    similarity_scores = []

    retrieved_similarity, retrieved_document = index_map[document].search(query_encoded, top_k)
    for q in range(len(query)):
        for k in range(top_k):
            similarity_scores.append(retrieved_similarity[q][k])
            documents_retrieved.append(f'''
                confidence : {retrieved_similarity[q][k]},
                source : {metadata[document][retrieved_document[q][k]]['source']},
                doc : {metadata[document][retrieved_document[q][k]]['chunk_text']}
            ''')
    return documents_retrieved, similarity_scores

def faiss_search_all(query, model, metadata, top_k=2):
    if isinstance(query, str):
        query = [query]
    query_encoded = model.encode(query, normalize_embeddings=True, convert_to_numpy=True)
    documents = []
    similarity = []

    for bases in Document:
        if bases == Document.ALL:
            continue
        sim, doc = index_map[bases].search(query_encoded, top_k)
        for q in range(len(query)):
            for k in range(top_k):
                similarity.append(sim[q][k])
                documents.append(f'''
                    confidence : {sim[q][k]},
                    source : {metadata[bases][doc[q][k]]['source']},
                    doc : {metadata[bases][doc[q][k]]['chunk_text']}
                ''')
    return documents, similarity

def faiss_search_all_top(query, model, metadata, top):
    # faisss_search_all but return top k matches
    documents, similarity = faiss_search_all(query, model, metadata, top)

    return sorted(zip(documents, similarity), reverse=True, key = lambda x : x[1])[:top]

question = "Can i take duck from local pond?"

print(faiss_search(question, model_faiss, Document.APEX, metadata, 1))

(["\n                confidence : 0.4602753221988678,\n                source : Supreme Court,\n                doc : The State including Respondents 11 to 13 shall restore the pond, develop and maintain the same as a recreational spot which will undoubtedly be in the best interest of the villagers.\nFurther it will also help in maintaining ecological balance and protecting the environment in regard to which this Court has repeatedly expressed its concern.\nSuch measures must begin at the grass-root level if they were to become the nation's pride. The appeal is accordingly allowed.\nThere shall be no order as to costs.\n            "], [np.float32(0.46027532)])


The cell below defines some prompt(s) which intend to serve different purposes according to need of user. And several combination(s) of prompts are then encapsulated as function(s) of different name(s) to be called for convinience.

In [18]:
documents = ""
prompt_reply_with_docs =  """<s>[INST]
You are a user-facing chat model working in a law firm.
You are given:
- A user question
- A set of document snippets retrieved from a legal database, each with a relevance score
Your task is to answer the user’s question using ONLY the information contained in the provided document snippets.

Rules:
- Use only facts, principles, or reasoning explicitly present in the provided snippets
- Do NOT rely on general legal knowledge or assumptions outside the documents
- Do NOT infer or reconstruct missing parts of documents
- If the provided documents do not directly address the legal issue raised in the question, apologize briefly and refuse to answer
- Do NOT mention the number of documents used
- Do NOT mention retrieval, embeddings, or confidence scores

Your reply must be:
- Concise
- Easy to understand for a casual user with no legal background
- Directly grounded in the provided documents

Question:
{user_question}

Documents:
{documents}
[/INST]""".format(user_question = question, documents = documents)

prompt_reply_rephrased = """<s>[INST]
You are a query rewriting engine for a Retrieval-Augmented Generation (RAG) system.

Your job:
- Rewrite the user's question into a concise, factual, search-oriented query
- Remove conversational language, opinions, and filler
- Preserve legal / technical terminology
- Do NOT answer the question
- Do NOT add new information
- Output ONLY the rewritten query

Think like a search engine, not a chatbot.

Question:
{user_question}

[/INST]""".format(user_question = question)

In [ ]:
reply = ""
documents = faiss_search(reply, model_faiss, Document.HIGH, metadata, 2)
prompt = """<s>[INST]
You are a user-facing chat model working in a law firm.
You are given:
- A user question
- A set of document snippets retrieved from a legal database, each with a relevance score
Your task is to answer the user’s question using ONLY the information contained in the provided document snippets.

Rules:
- Use only facts, principles, or reasoning explicitly present in the provided snippets
- Do NOT rely on general legal knowledge or assumptions outside the documents
- Do NOT infer or reconstruct missing parts of documents
- If the provided documents do not directly address the legal issue raised in the question, apologize briefly and refuse to answer
- Do NOT mention anything about existance of documents in your reply
- Do NOT mention retrieval, embeddings, or confidence scores

Your reply must be:
- Concise
- Easy to understand for a casual user with no legal background
- Directly grounded in the provided documents

Question:
{user_question}

Documents:
{documents}
[/INST]""".format(user_question = question, documents = documents)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=False
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

In [61]:
def ask_model(question):
    prompt = f"{question}"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=False
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def ask_with_retrieval(question, document=Document.APEX, top_k=2, cutoff_score=0.0, show_docs=False):
    """
    cutoff_score will later need to be passed to faiss search function :: TODO
    """
    documents = []

    if document == Document.ALL:
        documents, _ = faiss_search_all(question, model_faiss, metadata, top_k)
    else:
        documents, _ = faiss_search(question, model_faiss, document, metadata, top_k)

    if show_docs:
        for entries in documents:
            print(entries)

    prompt = prompt_reply_with_docs.format(user_question = question, documents = documents)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            do_sample=False
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

def ask_with_rephrasing(question, document=Document.APEX,top_k=2, cutoff_score=0.45, show_docs=False, compare=False):
    documents = []
    raw_scores = []
    if document == Document.ALL:
        documents, raw_scores = faiss_search_all(question, model_faiss, metadata, top_k)
    else:
        documents, raw_scores = faiss_search(question, model_faiss, document, metadata, top_k)

    query = prompt_reply_rephrased.format(user_question = question)
    inputs_1 = tokenizer(query, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_1 = model.generate(
            **inputs_1,
            max_new_tokens=300,
            do_sample=False
        )
    rephrased_query = tokenizer.decode(output_1[0], skip_special_tokens=True)
    retrieval = []
    scores = []
    if document == Document.ALL:
        retrieval, scores = faiss_search_all(rephrased_query, model_faiss, metadata, top_k)
    else:
        retrieval, scores = faiss_search(rephrased_query, model_faiss, document, metadata, top_k)
    if show_docs:
        for doc in retrieval:
            print(doc)
    if compare:
        print(f"scores of docs fetched by user query on faiss :: {raw_scores}")
        print(f"scores of docs fetched after rewriting by model :: {scores}")

    prompt = prompt_reply_with_docs.format(user_question = rephrased_query, documents = retrieval)
    inputs_2 = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_2 = model.generate(
            **inputs_2,
            max_new_tokens=300,
            do_sample=False
        )
    return tokenizer.decode(output_2[0], skip_special_tokens=True)

def get_full_document(id):
    pass


In [ ]:

ask_with_rephrasing(question, Document.ALL, 2, 0.34, False, True)

